# 종합실습2_1 따릉이

* 지금까지 배운 것을 총 복습 합니다.
* Data : 서울 공유 자전거
* 문제 : 2시간 후의 수요를 예측하고자 한다.

* OO시에서 에이블러 여러분에게 의뢰가 들어왔습니다.
* **공유 자전거 운영팀**에서는 공유자전거가 부족한 지역과 남는 지역에 대해서 판단하기 원합니다. 
* 2시간 전에 **공유자전거 수요량**을 예측할 수 있다면, 이동시켜서 남거나 부족한 문제를 해결할수 있다고 합니다.

![](https://cf.creatrip.com/original/blog/7376/kw69f1cy6ehvmmu9enuubth31zw5i7a7.jpg?d=1200&q=80&f=webp)

## 1.환경준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session
from keras.optimizers import Adam

* 함수 만들기

In [ ]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### (2) 데이터로딩

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/SeoulBikeData2.csv'
data = pd.read_csv(path)
data['DateTime'] = pd.to_datetime(data['DateTime'])
data.drop(['Visibility','Solar'], axis = 1, inplace = True)
data.head(10)

**변수설명**  

* DateTime : year-month-day hh:mi:ss
* Count : 시간대별 수요량
* Temperature : 온도(섭씨)
* Humidity : 습도(%)
* WindSpeed : 풍속(m/s)
* Rainfall - mm
* Snowfall - cm
* Seasons - Winter, Spring, Summer, Autumn
* Holiday - Holiday / No holiday
* FuncDay - Yes / No


In [ ]:
# 데이터 기간은 다음과 같습니다.
data.DateTime.min(), data.DateTime.max()

In [ ]:
# 14일 동안의 수요량을 살펴 봅시다.
temp = data[:24*14]
plt.figure(figsize = (20,8))
plt.plot('DateTime', 'Count', data = temp)
plt.grid()
plt.show()

## 2.데이터 준비

### (1) y 만들기 
* 2시간 이후의 수요량을 예측해야 합니다.

In [ ]:
data['y'] = data['Count'].shift(-2)
data.head()

In [ ]:
# 2칸을 앞당겼기 때문에 하위 2행의 y값에 NaN이 표시되어 있습니다.
data.tail()

In [ ]:
# 하위 2행은 삭제합니다.
# 하위 2행 제외하고 다시 붓기
data = data.iloc[:-2]

### (2) 데이터 정리
* 불필요한 변수 제거 : DateTime
* x, y  나누기

In [ ]:
target = 'y'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

In [ ]:
# 날짜 데이터 제거
x.drop('DateTime', axis = 1, inplace = True)
x.head()

### (3) NaN 조치

### (4) 가변수화

In [ ]:
cat_cols = ['Seasons','Holiday','FuncDay']
x = pd.get_dummies(x, columns = cat_cols, drop_first = True)

### (5) 데이터분할 : train : val

* 시계열 데이터이므로 시간의 흐름에 맞게 분할합시다.
    * 뒤에서 30일 : validaiton
    * 나머지 : train
    * 30일 : 시간단위 데이터이므로 24 * 30
* train_test_split : shuffle(뒤섞기) 옵션을 False로 하면 저장된 순서대로 자릅니다!


In [ ]:
i = 30 * 24
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = i, shuffle = False)

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(y_val)
plt.grid()
plt.show()

### (8) Scaling

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

## 3.모델링

* 히든레이어를 추가한 모델 두 개 이상을 생성한 후
* 성능을 비교하시오.
* 성능을 높이기 위해서 조절할 것들
    * 히든레이어 수
    * 히든레이어 노드수
    * epochs 수
    * learning_rate : 0.1 ~ 0.0001 사이에서 조정(예 Adam(learning_rate = 0.01))


### (1) 모델1

### (2) 모델2

## 4.무엇을 더 해볼 수 있을까요?